In [1]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import cvxopt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

In [2]:
def select_train(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

def select_test(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

In [3]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()

# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset1 = raw_dataset.copy()
test_dataset1.isna().sum()
test_dataset1 = test_dataset1.dropna()

# 2/ Test data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/test_dim2.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset2 = raw_dataset.copy()
test_dataset2.isna().sum()
test_dataset2 = test_dataset2.dropna()

# 3/ Test data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/test_dim3.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset3 = raw_dataset.copy()
test_dataset3.isna().sum()
test_dataset3 = test_dataset3.dropna()

In [4]:
list_comp = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']

list_var = ['input_width']

train_dataset = select_train(list_var, train_dataset)
test_dataset1 = select_test(list_var, test_dataset1)
test_dataset2 = select_test(list_var, test_dataset2)
test_dataset3 = select_test(list_var, test_dataset3)

In [5]:
Y = train_dataset.pop('memory')
X = train_dataset
X.tail()

,Model_name,input_shape,input_size,nb_layers,sum_activations,params
153,basic_model_448_37,"(1, 448, 448, 3)",602112,62,190772012,9215748
154,basic_model_112_11,"(1, 112, 112, 3)",37632,22,5828752,4989982
155,basic_model_480_9,"(1, 480, 480, 3)",691200,19,34142880,2556232
156,basic_model_320_15,"(1, 320, 320, 3)",307200,30,23250476,1610240
157,basic_model_90_4,"(1, 90, 90, 3)",24300,9,6439860,453340


In [6]:
Y_e1 = test_dataset1.pop('memory')
X_e1 = test_dataset1

Y_e2 = test_dataset2.pop('memory')
X_e2 = test_dataset2

Y_e3 = test_dataset3.pop('memory')
X_e3 = test_dataset3

In [7]:
X_train_model_names = X.pop('Model_name')
X_train_input_shapes = X.pop('input_shape')

In [8]:
X_test_model_names1 = X_e1.pop('Model_name')
X_test_input_shapes1 = X_e1.pop('input_shape')

X_test_model_names2 = X_e2.pop('Model_name')
X_test_input_shapes2 = X_e2.pop('input_shape')

X_test_model_names3 = X_e3.pop('Model_name')
X_test_input_shapes3 = X_e3.pop('input_shape')

In [9]:
y_test1 = Y_e1.values
y_test2 = Y_e2.values
y_test3 = Y_e3.values

In [10]:
polyn = PolynomialFeatures(degree=12)
xd = polyn.fit_transform(np.log(X.values))
clf_Ridge = linear_model.Ridge(alpha=0.1, normalize=True,max_iter=1e5)
A = clf_Ridge.fit(xd,Y.values)

In [11]:
y_pred1 = clf_Ridge.predict(polyn.fit_transform(np.log(X_e1.values)))
y_pred2 = clf_Ridge.predict(polyn.fit_transform(np.log(X_e2.values)))
y_pred3 = clf_Ridge.predict(polyn.fit_transform(np.log(X_e3.values)))

In [20]:
y_test = y_test1
y_pred = y_pred1
test_models = X_test_model_names1.values
test_input_shapes = X_test_input_shapes1.values

In [21]:
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD:', median_absolute_error(y_test, y_pred))

Explained variance :  0.9828582873548639
R2_value :  0.9826141418785374
Mean absolute error MAE : 140.5704266435843
Root Mean squared error RMSE : 168.72480014397206
Median absolute error MAD: 132.75511309932176


In [22]:
# Calculate mean absolute percentage error (MAPE)
errors = abs(y_pred-y_test)
mape = 100 * (errors / y_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE: ', round(max(mape), 2), '%.')
print('MAPE:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE:  12.98 %.
MAPE: 4.38 %.
Accuracy: 95.62 %.


In [23]:
import csv
with open('./measured_vs_predicted_Poly_dim1.csv', 'a', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'measured', 'predicted', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, y_test[i], y_pred[i],abs(y_pred[i]-y_test[i]), ((abs(y_pred[i]-y_test[i]))/y_test[i])*100])